## Comparing Regressions by RMSE

This script investigates the best regressions (by rmse) for all variables (without regard to how common they are in the data; that is, how much the variable is missing). 

I use the base variables: 
    'simple_return','age_in_months','borrower_rate','payment','amount_funded','term'

along with the best AIC variables from previous regression research. I chose this approach since there are far too many variables in general to try brute force. 

Note: I made a mistake by not dividing by the number of observations when calculating the RMSE. I have added this part in case I run this script again (it takes a long time to run ~several hours). However, note that the outputs below still reflect the mistake. I did correct the mistake in the database after the fact. 

In [3]:
import logging
import copy
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

from math import sqrt
from itertools import combinations
from db_conn import DBConn

In [4]:
#variables
BASE_VARS = ['simple_return','age_in_months','borrower_rate','payment',
             'amount_funded','term']
AIC_BEST1 = ["prior_prosper_loan_earliest_pay_off","all146","all071",
            "cap801","ale071","all005","rtr002","ale403"]    
AIC_BEST2 = ["rep071","ale075","iln071","rev130","bac071","bac078",
"ale804","all505","ale801","rtr071","bac022","bac023","iln007","rev081",
"rev086","rev085","rev084","rev002","all126","fil001","rep078","all153",
"fin001","rtl071","ale908","ale905","all092","all091","all155","all151",
"all086","all807","rev023","rev022","rep005","rep001","rtr023","all002",
"rev001","rtl905","ale005","rev115","rep501","rep904","rep905","rep906",
"rep901","bnk001","rtl001","rtl002","rtl005","fin801","ale002","bac084",
"bac081","bac005","bac001","bac002","rev128","rev129","aut071","rtr022",
"fico_score","all023","ale078","ale023","ale001","ale084","ale081","rep084",
"rep081","bac804","rtr001","bac801","rtr005","cru001","iln005","aut001",
"iln403","rev078","rev550","iln101","iln801","bac550","ale022","rev005"]

#### functions

In [5]:
def get_sql_data(varset, dataset):
    '''Given list of variables varset, get data from sql merged data. Dataset
    is either train, dev, or test.
    '''
    stmt = 'select %s from merged_%s where term<>12' %(','.join(varset), dataset)
    data = pd.read_sql(stmt, con)
    return data

def get_formula(varset):
    formula = 'simple_return ~ age_in_months + borrower_rate + payment + C(term)'
    for c in varset:
        #set formula    
        if CATS.category[CATS.name == c].values[0] == 'categorical':
            formula += '+C(%s)' %c
        else:
            formula += '+%s' %c
    return formula

In [6]:
def run_reg_combs(varset, data_train, data_dev, r=5):
    """Run regressions for combinations of varset fields on data."""
    top10_regs = pd.DataFrame(np.ones((10,4))*np.inf, 
                              columns=['rmse','aic','rsquared','tables'])
    for r in xrange(len(varset)):
        for i, cols in enumerate(combinations(varset, r)):
            data_t = data_train[list(cols)+BASE_VARS].dropna()
            data_d = data_dev[list(cols)+BASE_VARS].dropna()
            #regression
            formula = get_formula(cols)
            reg = smf.ols(formula=formula, data=data_t, missing='drop')
            model = reg.fit()            
            #prediction
            dev_pred = model.predict(data_d)
            #calculate RMSE
            rmse = sqrt(2*sum(
                        (dev_pred - data_d['simple_return'])**2
                          )/float(len(dev_pred))
                   )

            #print cols, ': ', rmse

            #check against best set, keep if good enough
            top10_regs.sort_values(by=['rmse','aic'],
                                  ascending=[True,True],
                                  inplace=True)
            if rmse < top10_regs.rmse.max():
                top10_regs = top10_regs[:-1]
                top10_regs = top10_regs.append(pd.DataFrame(
                                                    {'rmse': [rmse],
                                                     'aic': [model.aic],
                                                     'rsquared': [model.rsquared],
                                                     'tables': [model.summary().tables]},
                                               index=[str(cols)])
                                              )
                top10_regs.to_pickle('rmse_top10_regs.pkl')
    top10_regs.sort_values(by=['rmse','aic'],
                      ascending=[True,True],
                      inplace=True)
    return top10_regs
    

In [7]:
def run_reg_seq(core_vars, varset, data_train, data_dev, savefile='file.pkl'):                
    """Run regressions sequentially over vars in varset. The core_vars are vars
    in addition to BASE_VARS which will always be present."""
    """Run regressions for combinations of varset fields on data."""
    variables = copy.deepcopy(varset)
    core_v = copy.deepcopy(core_vars)
    core_data_t = data_train[core_vars+BASE_VARS].dropna()
    core_data_d = data_dev[core_vars+BASE_VARS].dropna()
    #base rmse
    core_formula = get_formula(core_v)
    core_reg = smf.ols(formula=core_formula, data=core_data_t, missing='drop')
    core_model = core_reg.fit()
    #prediction
    core_dev_pred = core_model.predict(core_data_d)
    #calculate RMSE
    core_rmse = sqrt(2*sum(
                    (core_dev_pred - core_data_d['simple_return'])**2
                          )/float(len(core_dev_pred))
                    )         
    while variables:
        best_rmse = np.inf   
        for field in variables:
            data_t = data_train[core_v+BASE_VARS+[field]].dropna()
            data_d = data_dev[core_v+BASE_VARS+[field]].dropna()
            #regression
            formula = get_formula(core_v + [field])
            reg = smf.ols(formula=formula, data=data_t, missing='drop')
            model = reg.fit()
            #prediction
            dev_pred = model.predict(data_d)
            #calculate RMSE
            rmse = sqrt(2*sum(
                        (dev_pred - data_d['simple_return'])**2
                          )/float(len(dev_pred))
                   )
            if rmse < best_rmse:
                best_rmse = rmse
                best_var = field
                best_model = model
                best_dev_pred = dev_pred
        if best_rmse < core_rmse:
            #add to core
            core_v += [best_var]
            variables = variables.remove(best_var)
            core_rmse = best_rmse
            core_model = best_model
            core_dev_pred = best_dev_pred
            #save res
            core_model.save(savefile)
        else:
            break  #break while loop
    return (core_v, core_rmse, core_model)

In [8]:
#setup logging
logging.basicConfig(filename='rmse_regs_compare'+'.log')
log = logging.getLogger()
log.setLevel('INFO')

In [9]:
#setup connection to db
eng = DBConn(username='',
             password='')    #need to fill in
con = eng.connect()
log.info('db connected')

In [10]:
#variable categories   
CAT_STMT = "select name, category from column_category"
CATS = pd.read_sql(CAT_STMT, con)        

In [11]:
#get train data
data_train = get_sql_data(BASE_VARS+AIC_BEST1+AIC_BEST2, 'train')
log.info('downloaded training data')
#get dev data
data_dev = get_sql_data(BASE_VARS+AIC_BEST1+AIC_BEST2, 'dev')
log.info('downloaded dev data')

In [12]:
print len(data_train)
print len(data_dev)

232261
29250


In [13]:
print len(data_train.dropna())
print len(data_dev.dropna())

9954
1327


In [14]:
CATS.category[CATS.name == 'prior_prosper_loan_earliest_pay_off'].values[0]

u'integer'

In [15]:
#identify best regs from core vars
log.info('starting run_reg_combs...')
top10_core = run_reg_combs(AIC_BEST1, data_train, data_dev, r=len(AIC_BEST1))

In [16]:
pd.options.display.max_colwidth = 500

In [17]:
print top10_core.ix[:,:-1]

                                                                                 aic  \
('cap801', 'ale071', 'all005', 'rtr002', 'ale403')                      25251.629525   
('cap801', 'ale071', 'all005', 'rtr002')                                25276.584760   
('ale071', 'all005', 'rtr002', 'ale403')                                25278.634965   
('all146', 'cap801', 'ale071', 'all005', 'rtr002', 'ale403')            25182.442050   
('ale071', 'all005', 'rtr002')                                          25324.127895   
('all146', 'cap801', 'ale071', 'all005', 'rtr002')                      25206.621701   
('all071', 'cap801', 'ale071', 'rtr002', 'ale403')                      25358.313786   
('all146', 'all071', 'cap801', 'ale071', 'rtr002', 'ale403')            25237.250451   
('all146', 'all071', 'cap801', 'ale071', 'all005', 'rtr002', 'ale403')  25182.767580   
('all071', 'cap801', 'ale071', 'all005', 'rtr002', 'ale403')            25231.938944   

                               

In [18]:
eval(top10_core.index[1])

('cap801', 'ale071', 'all005', 'rtr002')

In [19]:
#see if we can augment core vars regs
log.info('starting run_reg_seq...')
top_aug = pd.Series()
for i,vrs in enumerate(top10_core.index):
    top_aug = pd.concat([top_aug, pd.Series([run_reg_seq(list(eval(vrs)), AIC_BEST2, data_train,
                                              data_dev, 'rmse_best_seq_reg%d.pkl' %i)])], 
                        ignore_index=True)

#get most common vars set
#identify best regression for most common vars

In [20]:
top_aug.head()

0            ([cap801, ale071, all005, rtr002, ale403, rev550], 0.362422946085, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x7fd9e99e6610>)
1                    ([cap801, ale071, all005, rtr002, rev550], 0.362426945626, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x7fd9e9b26410>)
2                    ([ale071, all005, rtr002, ale403, rev550], 0.362434017683, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x7fd9eabd0b10>)
3    ([all146, cap801, ale071, all005, rtr002, ale403, rev550], 0.362454177712, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x7fd9ead565d0>)
4                            ([ale071, all005, rtr002, rev550], 0.362443752746, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x7fd9e9b59f50>)
dtype: object

In [21]:
top_aug.apply(lambda row: row[1])

0    0.362423
1    0.362427
2    0.362434
3    0.362454
4    0.362444
5    0.362458
6    0.362456
7    0.362467
8    0.362466
9    0.362460
dtype: float64

In [22]:
for row in top_aug:
    print '='*120
    print 'Fields: ', row[0]
    print 'RMSE: ', row[1]
    print '='*120
    print '\n'
    print row[2].summary()
    print '\n\n'

Fields:  ['cap801', 'ale071', 'all005', 'rtr002', 'ale403', 'rev550']
RMSE:  0.362422946085


                            OLS Regression Results                            
Dep. Variable:          simple_return   R-squared:                       0.536
Model:                            OLS   Adj. R-squared:                  0.536
Method:                 Least Squares   F-statistic:                 2.301e+04
Date:                Tue, 08 Nov 2016   Prob (F-statistic):               0.00
Time:                        06:18:14   Log-Likelihood:                -12360.
No. Observations:              199367   AIC:                         2.474e+04
Df Residuals:                  199356   BIC:                         2.485e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------